# App - 1

In [1]:
import nbimporter
import draw_chart
import check_account
import autologin
import watch_strat
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtChart import *
from PyQt5 import uic
from PyQt5 import QtChart as qc
import win32com.client
import ctypes
import pandas as pd
import os
import time
import numpy as np
import threading

Importing Jupyter notebook from draw_chart.ipynb
Importing Jupyter notebook from check_account.ipynb
Importing Jupyter notebook from autologin.ipynb
Importing Jupyter notebook from watch_strat.ipynb


In [2]:
# 대신증권 Cybos Plus 자동 로그인
auto_log = autologin.cp_login()
auto_log.login_virtual()

AttributeError: 'cp_login' object has no attribute 'connect'

In [2]:
# API 모듈 import
obj_codeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr') # 종목정보
obj_cpStatus = win32com.client.Dispatch('CpUtil.CpCybos') # 연결, 상태 정보
obj_cpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil') # 거래 초기화
obj_cpTd0311 = win32com.client.Dispatch('CpTrade.CpTd0311') # 거래 오브젝트
obj_stockMst = win32com.client.Dispatch("DsCbo1.StockMst") # 현재가
obj_stockChart = win32com.client.Dispatch("CpSysDib.StockChart") # 차트 데이터

In [3]:
# 상태 체크
def init_pluscheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('Cybos Plus - 정상 (관리자권한으로 실행된 프로세스입니다.)')
    else:
        print('Cybos Plus - 오류 (일반권한으로 실행됨. 관리자 권한으로 실행해 주세요)')
        return False
 
    # 연결 여부 체크
    if (obj_cpStatus.IsConnect == 0):
        print("Cybos Plus - PLUS 가 정상적으로 연결되지 않음. ")
        return False
    
    # 주문 관련 초기화
    if (obj_cpTrade.TradeInit(0) != 0):
        print("Cybos Plus - 주문 초기화 실패")
        return False
    else:
        print('Cybos Plus - 주문 초기화 완료')
    return True

init_pluscheck()

Cybos Plus - 정상 (관리자권한으로 실행된 프로세스입니다.)
Cybos Plus - 주문 초기화 완료


True

In [4]:
'''
 ================
| 종목 리스트 관련
| 리스트 구하고 저장
 ================
'''

codelist_market = obj_codeMgr.GetStockListByMarket(1) # 인자는 시장구분, 인자가 1 이면 거래소, 2 는 코스닥
codelist_kosdaq = obj_codeMgr.GetStockListByMarket(2) # 인자는 시장구분, 인자가 1 이면 거래소, 2 는 코스닥

len_market = len(codelist_market)
len_kosdaq = len(codelist_kosdaq)

print('거래소 종목 수 :', len_market)
print('코스닥 종목 수 :', len_kosdaq)

code_total = {} # 코스닥 종목만 저장

# {종목명 : 코드} 로 저장
for code in codelist_kosdaq:
    name = obj_codeMgr.CodeToName(code)
    code_total[name] = code

거래소 종목 수 : 1534
코스닥 종목 수 : 1356


In [5]:
'''
 ================
| 특징주 탐색
| 주가, 주문량, 거래량 등등으로
| 일반통신(RQ/RP)의 경우는 15초에 최대 60건수까지 가능. (초당 4 개)
| 실시간 통신(SB/PB)는 고객등급 차이 없이 동일한 서비스 제공. 주식종목 실시간을 400종목까지 등록가능.
 ================
'''

selected_list = {} # 모든 종목을 특정 조건에 따라 걸러 딕셔너리로 저장. {종목명 : 코드}
selected_list_names = [] # 걸러진 종목의 종목명만 따로 저장.

# 주가로 거르기
# 중복 처리하기
def selectby_items(max_cnt, caller):

    print('-- 탐색중 --')
    cnt = 0
    
    for name in code_total:
        cnt += 1
        # 대기 창 안 뜨도록 내부적으로 대기
        if(cnt % 4 == 0):
            time.sleep(1.5)
            pass
        
        # 현재가
        obj_stockMst.SetInputValue(0, code_total[name])
        req_result = obj_stockMst.BlockRequest()
        
        # 15 초 당 60 건 초과 시 처리
        if(req_result == 4):
            print('과다 요청. 대기')
        
        # 현재가 통신 및 통신 에러 처리 
        rqStatus = obj_stockMst.GetDibStatus()
        rqRet = obj_stockMst.GetDibMsg1()
        #print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()

        # 현재가 정보 조회
        #code = obj_stockMst.GetHeaderValue(0)  #종목코드
        #name= obj_stockMst.GetHeaderValue(1)  # 종목명
        t = obj_stockMst.GetHeaderValue(4)  # 시간
        cprice = obj_stockMst.GetHeaderValue(11) # 현재가
        diff = obj_stockMst.GetHeaderValue(12)  # 전일 대비
        open = obj_stockMst.GetHeaderValue(13)  # 시가
        high = obj_stockMst.GetHeaderValue(14)  # 고가
        low = obj_stockMst.GetHeaderValue(15)   # 저가
        offer = obj_stockMst.GetHeaderValue(16)  #매도호가
        bid = obj_stockMst.GetHeaderValue(17)   #매수호가
        vol = obj_stockMst.GetHeaderValue(18)   #거래량
        vol_value = obj_stockMst.GetHeaderValue(19)  #거래대금
        offer_vol = 0 # 10 차 총 매도 잔량
        bid_vol = 0 # 10 차 총 매수 잔량
        
        # 10차 총 매도수 잔량계산
        for i in range(10):
            #rtMst.offer.append(self.objStockMst.GetDataValue(0, i))  # 매도호가
            #rtMst.bid.append(self.objStockMst.GetDataValue(1, i) ) # 매수호가
            offer_vol += obj_stockMst.GetDataValue(2, i)  # 매도호가 잔량
            bid_vol += obj_stockMst.GetDataValue(3, i) # 매수호가 잔량
        
        # -----------------
        # 특정 조건으로 거르기
        # ------------------
        
        # 거래량으로 거르기
        if(vol < 2000):
            continue
        
        # 거래대금으로 거르기
        if(vol_value < 20000):
            continue
        
        # 호가 정보로 거르기
        if(bid_vol < offer_vol):
            continue
        
        # 과거 5일 데이터 가져와서 지표계산
        obj_stockChart.SetInputValue(0, code_total[name])   #종목 코드
        obj_stockChart.SetInputValue(1, ord('2')) # 개수로 조회
        obj_stockChart.SetInputValue(4, 5) # 최근 100일 치
        obj_stockChart.SetInputValue(5, [0,2,3,4,5,8]) #날짜,시가,고가,저가,종가,거래량
        obj_stockChart.SetInputValue(6, ord('D')) # '차트 주가 - 일간 차트 요청
        obj_stockChart.SetInputValue(9, ord('1')) # 수정주가 사용
        
        req_result = obj_stockChart.BlockRequest()
        if(req_result == 4):
            print('과다 요청. 대기')
        
        len_day = obj_stockChart.GetHeaderValue(3)
        value_mean = 0
        for i in range(len_day):
            value_mean += obj_stockChart.GetDataValue(4, i)
        value_mean = int(value_mean / 5) # 5 일 평균 종가
        
        # 현재가로 거르기
        if(cprice < value_mean):
            continue
        
        # 다 걸러졌으면 관심종목에 추가
        if(len(selected_list) < max_cnt):
            print('> 선택 종목 :', name)
            selected_list[name] = code_total[name]
            caller.listWidget.addItem(name) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            if(len(selected_list) == max_cnt):
                break
        # 최대 저장크기 넘어가면 걍 종료
        else:
            break
    
    print('-- 탐색완료 --')
    selected_list_names = list(selected_list.keys()) 
    caller.label_stat.setText("{} 개 종목 탐색 완료.".format(max_cnt)) # 상태 텍스트에 표시


In [6]:
'''
 ================
| 차트 (1 시간 전 ~ 현재. 1 분봉) 그리는 데 사용될 class
 ================
'''
class Chart():
    def __init__(self):
        pass

    # 차트 요청 - 1 분봉으로 1 시간 분량 (1 시간 이상 데이터가 없으면 처리)
    def req_chartdata(self, code, num_req=60):
        obj_stockChart.SetInputValue(0, code)  # 종목코드
        obj_stockChart.SetInputValue(1, ord('2'))  # 갯수로 요청
        obj_stockChart.SetInputValue(4, num_req)  # 최근 60 분
        obj_stockChart.SetInputValue(5, [1, 2, 3, 4, 5, 8])  # tOHLCv
        obj_stockChart.SetInputValue(6, ord('m'))  # '차트 주기 - 분봉
        obj_stockChart.SetInputValue(9, ord('1'))  # 수정주가 사용
        obj_stockChart.BlockRequest()

        len_stick = obj_stockChart.GetHeaderValue(3)

        times = []
        order = []
        opens = []
        highs = []
        lows = []
        closes = []
        means = []
        vols = []
        for i in range(len_stick):
            #times.append(obj_stockChart.GetDataValue(0,i))
            order.append(i+1)
            opens.append(obj_stockChart.GetDataValue(1, i))
            highs.append(obj_stockChart.GetDataValue(2, i))
            lows.append(obj_stockChart.GetDataValue(3, i))
            closes.append(obj_stockChart.GetDataValue(4, i))
            vols.append(obj_stockChart.GetDataValue(5, i))
        
        data = [order, opens, highs, lows, closes, vols]
        data = pd.DataFrame(data).transpose()
        ma5 = data[4].rolling(window=5).mean()
        data.insert(5, 'ma5', ma5)

        return data

In [7]:
'''
 ================
| 주문 관련
 ================
'''
class Trade():
    
    def __init__(self, acc_num, caller):
        self.account_num = acc_num
        self.caller = caller
    
    # 현금 매수
    def buy(self, item, vol, price):
        accFlag = obj_cpTrade.GoodsList(self.account_num, 1)  # 주식상품 구분
        print(self.account_num, accFlag[0])
        obj_cpTd0311.SetInputValue(0, 2) # 첫번째인자 : 주문종류코드, 두번째인자 : 매도(1)/매수(2)
        obj_cpTd0311.SetInputValue(1, self.account_num) # 계좌, 계좌번호 값
        obj_cpTd0311.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
        obj_cpTd0311.SetInputValue(3, item) # 종목코드, 코드 값 (대신증권 주식)
        obj_cpTd0311.SetInputValue(4, vol) # 주문 수량
        obj_cpTd0311.SetInputValue(5, price) # 주문 단가
        obj_cpTd0311.SetInputValue(7, 0)   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
        obj_cpTd0311.SetInputValue(8, "01")   # 주문호가 구분코드 - 01: 보통

        # 요청
        obj_cpTd0311.BlockRequest() # 

        # 확인은 conclusion 클래스로
        if obj_cpTd0311.GetDibStatus() != 0:
            print("통신상태", obj_cpTd0311.GetDibStatus(), obj_cpTd0311.GetDibMsg1())
        else:
            print("예약주문 성공, 예약번호 #")


    # 현금 매도
    def sell(self, item, vol, price):
        accFlag = obj_cpTrade.GoodsList(self.account_num, 1)  # 주식상품 구분
        print(self.account_num, accFlag[0])
        obj_cpTd0311 = win32com.client.Dispatch("CpTrade.CpTd0311")
        obj_cpTd0311.SetInputValue(0, "1")   #  1: 매도
        obj_cpTd0311.SetInputValue(1, self.account_num )   #  계좌번호
        obj_cpTd0311.SetInputValue(2, accFlag[0])   #  상품구분 - 주식 상품 중 첫번째
        obj_cpTd0311.SetInputValue(3, item)   #  종목코드 - A003540 - 대신증권 종목
        obj_cpTd0311.SetInputValue(4, vol)   #  매도수량
        obj_cpTd0311.SetInputValue(5, price)   #  주문단가
        obj_cpTd0311.SetInputValue(7, "0")   #  주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
        obj_cpTd0311.SetInputValue(8, "01")   # 주문호가 구분코드 - 01: 보통

        # 매도 주문 요청
        obj_cpTd0311.BlockRequest()
        
        self.caller.lookup_data.misscodeList.append(item) # 매도주문 넣으면 일단 빼기

        rqStatus = obj_cpTd0311.GetDibStatus()
        rqRet = obj_cpTd0311.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()

In [8]:
'''
 ============
| Designer 로 미리 만든 UI Form 불러오기
| UI 이벤트 함수 정의
 ============
'''

form = uic.loadUiType('C:\\Users\\김민수\\Desktop\\Trader.ui')[0] # UI 로드

class myWindow(QMainWindow, form):
    def __init__(self):
        super().__init__()
        self.setupUi(self) # UI 셋팅
        self.cur_selected_item = None # 현재 선택한 종목을 종목명으로 저장; 첫 리스트로 초기화
        self.obj_trade = Trade(obj_cpTrade.AccountNumber[0], self) # 주문 관련 객체 생성
        self.obj_chartdata = Chart() # 차트 데이터 받는 객체 생성
        self.lookup_data = check_account.Lookup_account() # 실시간 계좌조회 오브젝트
        self.obj_watch = watch_strat.Watch(obj_codeMgr) # 실시간 감시 오브젝트
        self.label_stat.setText("") # 상태 텍스트
        self.label_time.setText(QDateTime.currentDateTime().toString(Qt.DefaultLocaleLongDate)) # 현재 시간 표시
        self.set_timer_time() # 시간 실시간 업뎃
        self.set_timer_account() # 계좌잔고 실시간 업뎃
        self.set_timer_sell() # 자동매매 실시간 판단
        self.chart_candlestick.setChart(QChart()) # 차트뷰(중요). 초기화 필요없지만 일단 해놓음
        self.jango_data = {} # 계좌잔고 데이터 저장
        self.total_data = {} # 계좌 종합 데이터 저장
        self.labels = [self.label, self.label_2, self.label_3, self.label_4, self.label_5] # 최대 갯수 제한해서 매매할것이므로 N 개.
        self.labels_miss = [self.label_6, self.label_7, self.label_8, self.label_9, self.label_10]

        
        # -- UI 이벤트 콜백함수 등록 --
        self.btn_search.clicked.connect(self.btn_search_clicked) # 탐색 버튼 이벤트
        self.btn_buy.clicked.connect(self.btn_buy_clicked) # 매수 버튼 이벤트
        self.btn_watch.clicked.connect(self.btn_watch_clicked) # 감시 버튼 이벤트
        self.btn_watch_stop.clicked.connect(self.btn_watch_stop_clicked) # 감시 종료 버튼 이벤트
        self.listWidget.clicked.connect(self.clicked_list) # 리스트에서 종목 클릭시 이벤트
    
    # -----------
    # - 이벤트 함수  
    # ---------
    # 탐색
    def btn_search_clicked(self):
        cnt_search = int(self.edit_cnt.text())
        self.label_stat.setText("{} 개 종목 탐색중...".format(cnt_search))
        global selected_list # 선택 종목 글로벌 변수
        self.listWidget.clear() # 종목 리스트 위젯 초기화
        selected_list = {} # 종목 리스트(딕셔너리) 초기화
        t_search = threading.Thread(target=selectby_items, args=(cnt_search, self)).start() # 종목탐색 실시. 종목탐색은 별도의 스레드에서 실행. ui 업데이트도 스레드에서
    
    # 매수
    def btn_buy_clicked(self):
        #QMessageBox.about(self, "message", "clicked")
        item = selected_list[self.cur_selected_item] # 매수할 종목
        #print(item)
        obj_stockMst.SetInputValue(0, item)
        req_result = obj_stockMst.BlockRequest() # 요청
        cprice = obj_stockMst.GetHeaderValue(11) # 현재가 get
        #self.obj_trade.buy(item, 10, cprice) # 매수!
        t_buy = threading.Thread(target=self.obj_trade.buy, args=(item, 10, cprice)).start() # 종목탐색 실시. 종목탐색은 별도의 스레드에서 실행. ui 업데이트도 스레드에서
    
    # 종목 리스트 클릭 시 - 차트그리고 현재 선택종목 업데이트
    def clicked_list(self):
        self.cur_selected_item = self.listWidget.currentItem().text() # 현재 선택 종목 갱신
        self.label_cur_item.setText('현재 종목 : {}'.format(self.cur_selected_item))
        self.draw_candlechart() # 차트 그리기
    
    # -----------
    # - 기타 유틸 함수 
    # ---------
    
    # 시간 표시 함수
    def set_timer_time(self):
        self.timer_date = QTimer()
        self.timer_date.timeout.connect(self.update_time) # 시간 업데이트 함수 연결
        self.timer_date.setSingleShot(False)
        self.timer_date.start(1000)
    
    def update_time(self):
        self.label_time.setText(QDateTime.currentDateTime().toString(Qt.DefaultLocaleLongDate)) # 현재 시간 표시
        
    # 주식 계좌 조회 전용 타이머 - 3 초 주기
    def set_timer_account(self):
        self.timer_account = QTimer()
        self.timer_account.timeout.connect(self.account_update) # 잔고 조회 함수 연결
        self.timer_account.setSingleShot(False)
        self.timer_account.start(3000)
        
    # 잔고 조회
    def account_update(self):
        self.jango_data, cur, self.total_data = self.lookup_data.get_accountdata() # 주식 잔고 데이터, 실시간 수신하고있는 데이터
        self.label_accountName.setText('계좌 : {}'.format(self.total_data['계좌명']))
        self.label_profit.setText('수익률 : {} %'.format(round(self.total_data['수익률'], 2)))
        
        # 텍스트들 지워서 초기화
        for i in range(len(self.labels)):
            self.labels[i].setText('')
        for i in range(len(self.labels_miss)):
            self.labels_miss[i].setText('')
            
        i = 0
        i_miss = 0
        
        # 미체결 건 표시
        for key in self.lookup_data.diOrderList:
            self.labels_miss[i_miss].setText("{} : {}".format(self.lookup_data.diOrderList[key].name, self.lookup_data.diOrderList[key].orderDesc)) # 
            i_miss += 1
        
        for code, value in self.jango_data:
            # 만약에 최대 label 이상 뽑으면 걍 넘어감
            if(i == 5):
                break
            
            # 만약 예약매도가 걸려있으면 표시 안함
            if(code in self.lookup_data.diOrderList):
                pass
            else:
                profit = value['수익률']
                #profit_val = value['수익률'] * value['잔고수량']
                profit_val = int(value['평가손익'] / 1000) # 총 이익금
                self.labels[i].setText("{} : {} %,  잔고 : {}, 이익금 : {}".format(value['종목명'], round(profit, 2), value['잔고수량'], profit_val)) # 종목명, 이익률, 잔고, 이익금
                i += 1
            
    
    # 캔들 차트 그리기 # 차트 그리기 (designer 에서 promote 하여 사용!!)
    def draw_candlechart(self):
        
        # ------ 캔들스틱 -----------
        # 캔들스틱 series
        series = QCandlestickSeries()
        series.setIncreasingColor(Qt.red)
        series.setDecreasingColor(Qt.blue)
        
        # 이평선 series
        ma5_series = qc.QLineSeries()  # 5-days average data line
        tm = []  # stores str type data
        
        # 거래량 series
        series_vol = QBarSeries()
        set_vol = QBarSet('volume')
        
        # in a loop,  series and ma5 append corresponding data
        chart_data = self.obj_chartdata.req_chartdata(selected_list[self.cur_selected_item]) # 현재 선택 종목의 차트 요청
                    
        for idx, data in chart_data.iterrows():
            data = data.values
            series.append(QCandlestickSet(data[1], data[2], data[3], data[4]))
            ma5_series.append(QPointF(data[0], data[5]))
            set_vol.append(data[6])
            tm.append(str(data[0]))
                    
        chart = QChart()

        chart.addSeries(series)  # candle
        chart.addSeries(ma5_series)  # ma5 line

        chart.setAnimationOptions(QChart.SeriesAnimations)
        chart.createDefaultAxes()
        chart.legend().hide()

        chart.axisX(series).setCategories(tm)
        chart.axisX(ma5_series).setVisible(True)
        self.chart_candlestick.setChart(chart)
        
        # -------- 거래량 차트 ---------
        series_vol.append(set_vol)
        chart_vol = QChart()
        chart_vol.addSeries(series_vol) # volume
        chart_vol.setAnimationOptions(QChart.SeriesAnimations)
        chart_vol.createDefaultAxes()
        chart_vol.legend().hide()

        chart_vol.axisX(series_vol).setCategories(tm)
        self.chart_vol.setChart(chart_vol)
    
    
    # 자동매매 타이머 셋팅 - 일단 5 초
    def set_timer_sell(self):
        self.timer_sell = QTimer()
        self.timer_sell.timeout.connect(self.auto_sell) # 자동매매 함수 연결
        self.timer_sell.setSingleShot(False)
        self.timer_sell.start(5000)
    
    # 주기적으로 체크하면서 자동으로 매매하는 로직
    def auto_sell(self):
        # ---- 테스트 -----------
        self.miss_trade()
        
        for code, value in self.jango_data:
            
            # 만약 예약매도가 걸려있으면 표시 안함
            if(code in self.lookup_data.diOrderList):
                print('매도 미체결 :', code)
                continue
            
            profit = value['수익률']
            
            # 3 % 이익보면 익절 자동매매
            if(profit > 3):
                #self.obj_trade.sell(value['종목코드'], value['잔고수량'], value['현재가'])
                t_sell = threading.Thread(target=self.obj_trade.sell, args=(value['종목코드'], value['잔고수량'], value['현재가'])).start() # 종목탐색 실시. 종목탐색은 별도의 스레드에서 실행. ui 업데이트도 스레드에서
            # 2 % 손해보면 손절 자동매매
            elif(profit < -2):
                #self.obj_trade.sell(value['종목코드'], value['잔고수량'], value['현재가'])
                t_sell = threading.Thread(target=self.obj_trade.sell, args=(value['종목코드'], value['잔고수량'], value['현재가'])).start() # 종목탐색 실시. 종목탐색은 별도의 스레드에서 실행. ui 업데이트도 스레드에서
                
    #-------------
    # ------테스트
    #-----------
    
    # 미체결 주문 정보 업데이트
    def miss_trade(self):
        self.lookup_data.Reqeust5339()
    
    # 전략감시 이벤트
    def btn_watch_clicked(self):
        self.obj_watch.listAllStrategy()
        self.obj_watch.watch_stg()
        
        for item in self.obj_watch.dataStg:
            self.listWidget.addItem(item['종목명']) # 탐색한 리스트 웨젯에 표시 - 하나씩 추가
            selected_list[item['종목명']] = item['code']
        #print(self.obj_watch.dataStg)
        
    # 감시해제 이벤트
    def btn_watch_stop_clicked(self):
        self.obj_watch.btnExit_clicked()

In [9]:
'''
 ============
| UI Form
| Main
 ============
'''

if __name__ == '__main__':
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    else:
        print('QApplication instance already exists: %s' % str(app))

    window = myWindow()
    #window.setGeometry(100, 100, 500, 300)
    #window.setWindowTitle("PyQT Tuts!")
    window.show()
    app.exec_()
    app.exit()

335021511 10
통신상태 0 3700 모의투자 조회완료 되었습니다(cii.cif.astbln.selcnsbaseaccpt)
A002230 현금신용 현금
A003310 현금신용 현금
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
종목검색 전략수: 1
{'전략명': '단타1', 'ID': 'xUK1IzWmSB6/EjK0OOGeUQ', '전략등록일시': '2019/07/31 21:38', '작성자필명': '', '평균종목수': 7, '평균승률': 0.4067, '평균수익': -0.0034000000000000002}
[Cp5339] 미체결 데이터 조회 시작
[Cp5339] 수신 개수  0
[Reqeust5339]미체결 개수  0
종목검색 전략수: 1
{'전략명': '단타1', 'ID': 'xUK1IzWmSB6/EjK0OOGeUQ', '전략등록일시': '2019/07/31 21:38', '작성자필명': '', '평균종목수': 7, '평균승률': 0.4067, '평균수익': -0.0034000000000000002}
선택전략 : 단타1
0
검색된 종목수: 4 전체종목수: 4 검색시간: 07/31 21:38:54
{'code': 'A045660', '종목명': '에이텍'}
{'code': 'A058630', '종목명': '엠게임'}
{'code': 'A123040', '종목명': '엠에스오토텍'}
{'code': 'A290120', '종목명': '대유에이피'}
검색전략: xUK1IzWmSB6/EjK0OOGeUQ 전략명: 단타1 검색종목수: 4
감시일련번호 10
전략감시 시작 요청  전략 ID: xUK1IzWmSB6/EjK0OOGeUQ 감시일련번호 10
전략감시상태: 감시중
[Cp5339] 미체결 데이터 조회 시작
[Cp5339] 수신 개수  0
[Reqeust5339]미체결 개수  0
[Cp5339] 미체결 데이터 조회 시작
[Cp5339] 수신 개수  0
[Reqeust5339]미체결 개수  0
[Cp5339] 미체결 데이터 조회 시작
[Cp